<a href="https://colab.research.google.com/github/jihyoung-lee/AI-study/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# RandomForestClassifier 클래스로 화이트와인 분류하기
#와인 데이터셋을 판다스로 불러오고 훈련 세트와 테스트 세트로 나눔
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
#cross_validate 함수를 사용하여 교차 검증 수행
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# return_train_score 매개변수를 True로 지정하면 검증점수와 훈련세트 점수도 같이 반환(과대적합을 파악하기 용이)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) 

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
# 각 결정 트리의 특성 중요도를 취합한 랜덤 포레스트의 특성중요도 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
# 부트스트랩 샘플에 포함되지 않고 남는 샘플(OOB)을 이용하여 부트스트랩 샘플로 훈련한
#결정트리를 평가할 수 있음
#oob_score를 True로 지정해주면 됨
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [5]:
# 사이킷런에서 제공하는 엑스트라 트리는 ExtraTreesClassifier 
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

#랜덤 포레스트와 비슷한 결과를 얻을 수 있음
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 엑스트라 트리도 마찬가지로 특성 중요도를 제공함
et.fit(train_input, train_target)
print(et.feature_importances_)

0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]


In [6]:
# 사이킷런에서 제공하는 GradientBoostingClassifier 을 사용해 데이터셋 교차 검증 점수 확인
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [7]:
# 결정 트리 개수를 500개로 늘림
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#특성 중요도 랜덤 포레스트보다 일부 특성(당도)에 더 집중함
gb.fit(train_input, train_target)
print(gb.feature_importances_)

0.9464595437171814 0.8780082549788999
[0.15872278 0.68010884 0.16116839]


In [8]:
#사이킷런의 히스토그램 기반 그레이디언트 부스팅 클래스는 HistGradientBoostingClassifier
#이 클래스를 사용하려면 sklearn.experimental패키지 아래에 있는  enable_hist_gradient_boosting 을 임포트해주어야함
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [9]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [10]:
# 테스트 세트에서의 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)
hgb.score(test_input, test_target)

[0.05969231 0.20238462 0.049     ]


0.8723076923076923

In [ ]:
#XGBoost 라이브러리
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

In [ ]:
#LightGBM 라이브러리
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))